Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

---

前回の`2_fullyconnected.ipynb`でロジスティック回帰とニューラルネットワークモデルでトレーニングを行いました。

このアサインメントのゴールは、正則化(regularization)のテクニックを深掘りします

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

ロジスティック回帰とニューラルネットワークでL2正則化を追加してみてください。L2正則化は、ノルム(norm)として、コストにペナルティを追加します。
TensorFlowでは、`nn.l2_loss(t)`として計算します。正則化をすることで、バリデーションとテスト精度は向上するでしょう。

---

In [5]:
# ロジスティック回帰の宣言（Optimizer: SGD)
batch_size = 128
l2_lambda = 5e-4

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    # トレーニングデータは、placeholderで宣言し、トレーニング実行時に
    # ミニバッチとしてデータを渡せるようにします
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数 (Valriables)
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # L2正則化　(重み減衰)
    l2 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    loss += l2_lambda * l2
    
    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
# ロジスティック回帰の実行
# 正則化前: Last Minibatch accuracy: 78.9%, Validation accuracy: 78.7%, Test accuracy: 86.4%
# 正則化後: Last Minibatch accuracy: 80.5%, Validation accuracy: 81.0%, Test accuracy: 88.3%
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 20.843662
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 1000: 1.777348
Minibatch accuracy: 78.9%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.015113
Minibatch accuracy: 82.8%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 0.984953
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Test accuracy: 88.3%


In [7]:
# ニューラルネットワークの定義(L2正則化)
# input:      [batch_size, 784(28x28)]
# hidden1: weights(784x1024) + bias(1024) => relu
# output:   weights(1024x10) + bias(10) => softmax, cross_entropy
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数 (Valriables)
    hidden1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    hidden1_biases = tf.Variable(tf.zeros([hidden_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    # 隠れ層の計算
    hidden1_Wx_plus_b = tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases
    hidden1_activation = tf.nn.relu(hidden1_Wx_plus_b)
    
    # 出力層の計算
    logits = tf.matmul(hidden1_activation, output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # L2正則化　(重み減衰)
    l2 = (tf.nn.l2_loss(hidden1_weights) + tf.nn.l2_loss(hidden1_biases)
                 + tf.nn.l2_loss(output_weights) + tf.nn.l2_loss(output_biases))
    loss += l2_lambda * l2
    
    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)

In [8]:
# ニューラルネットワークの実行(L2正則化)
# L2正則化前: Last Minibatch accuracy: 83.6%, Validation accuracy: 82.8%, Test accuracy: 89.8%
# L2正則化後: Last Minibatch accuracy: 83.6%, Validation accuracy: 84.3%, Test accuracy: 91.5%
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 456.293457
Minibatch accuracy: 9.4%
Validation accuracy: 29.8%
Minibatch loss at step 1000: 101.249725
Minibatch accuracy: 78.1%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 57.148479
Minibatch accuracy: 87.5%
Validation accuracy: 82.9%
Minibatch loss at step 3000: 34.430706
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Test accuracy: 90.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

オーバーフィッティングの極端な例をデモンストレートしてみましょう。トレーニングデータを少数のバッチで制限するとどうなるでしょうか？

---

In [9]:
# ニューラルネットワーク (オーバーフィッティング)
# input:      [batch_size, 784(28x28)]
# hidden1: weights(784x1024) + bias(1024) => relu
# output:   weights(1024x10) + bias(10) => softmax, cross_entropy
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # 変数 (Valriables)
    hidden1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    hidden1_biases = tf.Variable(tf.zeros([hidden_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))

    # トレーニングの計算 (Training computation)
    # 隠れ層の計算
    hidden1_Wx_plus_b = tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases
    hidden1_activation = tf.nn.relu(hidden1_Wx_plus_b)
    
    # 出力層の計算
    logits = tf.matmul(hidden1_activation, output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)

In [10]:
# ニューラルネットワークの実行（オーバーフィッティング)
# L2正則化前: Last Minibatch accuracy: 83.6%, Validation accuracy: 82.8%, Test accuracy: 89.8%
# L2正則化後: Last Minibatch accuracy: 83.6%, Validation accuracy: 84.3%, Test accuracy: 91.5%
# Overfitting: Last Minibatch accuracy: 100.0%, Validation accuracy: 79.7%, Test accuracy: 87.3%
num_steps = 3001
n_samples = 3000 # トレーニングを3000件に制限する

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (n_samples - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 320.707336
Minibatch accuracy: 12.5%
Validation accuracy: 20.2%
Minibatch loss at step 1000: 0.596311
Minibatch accuracy: 99.2%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Test accuracy: 87.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

ニューラルネットワークの隠れ層にドロップアウトを追加してください。ドロップアウトはトレーニング時のみだけ実行してください。精度の評価のときにもドロップアウトをすると確率的な予測になってしまいます。
TensorFlowでは、`nn.dropout()`によりドロップアウトをします。

オーバーフィッティングの状態がどのようになるでしょうか？

---

In [11]:
# ニューラルネットワーク (オーバーフィッティング + ドロップアウト)
# input:      [batch_size, 784(28x28)]
# hidden1: weights(784x1024) + bias(1024) => relu
# output:   weights(1024x10) + bias(10) => softmax, cross_entropy
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

    # 入力データ (Input data)
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder(tf.float32) # ドロップアウトしない率

    # 変数 (Valriables)
    hidden1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    hidden1_biases = tf.Variable(tf.zeros([hidden_size]))
    output_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    
    # トレーニングの計算 (Training computation)
    # 隠れ層の計算
    hidden1_Wx_plus_b = tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases
    hidden1_activation = tf.nn.relu(hidden1_Wx_plus_b)
    # ドロップアウト
    hidden1_activation_drop = tf.nn.dropout(hidden1_activation, keep_prob)
    # 出力層の計算
    logits = tf.matmul(hidden1_activation_drop, output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # 最適化(Optimizer.)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # トレーニング、バリデーション、テストデータに対しての予測
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases), output_weights) + output_biases)

In [12]:
# ニューラルネットワークの実行（オーバーフィッティング + ドロップアウト)
# L2正則化前: Last Minibatch accuracy: 83.6%, Validation accuracy: 82.8%, Test accuracy: 89.8%
# L2正則化後: Last Minibatch accuracy: 83.6%, Validation accuracy: 84.3%, Test accuracy: 91.5%
# Overfitting: Last Minibatch accuracy: 100.0%, Validation accuracy: 79.7%, Test accuracy: 87.3%
# ドロップアウト: Last Minibatch accuracy: 98.4%, Validation accuracy: 81.8%, Test accuracy: 89.3%
num_steps = 3001
n_samples = 3000 # トレーニングを3000件に制限する

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("変数を初期化しました")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (n_samples - batch_size)

        # ミニバッチの作成
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5 }
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

変数を初期化しました
Minibatch loss at step 0: 538.235352
Minibatch accuracy: 12.5%
Validation accuracy: 20.9%
Minibatch loss at step 1000: 2.984866
Minibatch accuracy: 98.4%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 0.527159
Minibatch accuracy: 99.2%
Validation accuracy: 81.3%


KeyboardInterrupt: 

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 

## 隠れ層1層 + Adam + 10,000 epochs + batch size 128

|hidden1|Last Minibatch accuracy | Validation accuracy | Test accuracy
|---------|---------|---------|---------
| 100   | 79.7% | 79.1% | 86.8%
| 500   | 84.4% | 84.1% | 91.1%
| 1000 | 86.7% | 84.6% | 91.6%
| 3000 | 89.1% | 85.8% | 92.4%

## 隠れ層2層 + Adam + 10,000 epochs + batch size 128

|hidden1|hidden2| Last Minibatch loss | Last Minibatch accuracy | Validation accuracy | Test accuracy
|------- |------|----------------|---------|---------|---------
| 100  | 100 |  1.159447 | 73.4% | 76.6% | 84.6%
| 500  | 100 |  0.909591 | 65.6% | 69.0% | 77.1%
| 500  | 500 | 17.108561 | 90.6% | 84.5% | 91.0%
| 1000 | 100 |  1.130891 | 60.9% | 66.0% | 74.8%
| 1000 | 500 | 13.234963 | 83.6% | 81.7% | 88.4%
| 1000 | 1000| 29.798737 | 91.4% | 85.7% | 92.2%

## 隠れ層3層 + Adam + 30,000 epochs + batch size 128

|hidden1|hidden2|hidden3| Last Minibatch loss | Last Minibatch accuracy | Validation accuracy | Test accuracy
|------ |-------|------|----------------|-----------|---------|---------
| 100 | 100 | 100 | 0.445341  | 85.2% | 82.6% | 89.7%
| 500 | 100 | 100 | 0.759523  | 74.2% | 69.9% | 78.7%
| 500 | 300 | 100 | 1.501994  | 44.5% | 41.1% | 46.9%
| 500 | 500 | 100 | 2.304023  | 12.5% | 10.0% | 10.0%
| 500 | 500 | 500 | 12.310343 | 96.9% | 84.5% | 90.7%

In [15]:
# ニューラルネットワーク 
batch_size = 256
hidden1 = 1000
hidden2 = 1000

num_steps = 100000 + 1
# lr_params = [0.01, 0.001, 0.0001, 0.00001]
# hidden1_params = [100, 500, 1000, 3000]
keep_prob_params = [1.0, 0.9]

for keep_prob_val1 in keep_prob_params:

    graph = tf.Graph()
    with graph.as_default():

        # 入力データ (Input data)
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        keep_prob1 = tf.placeholder(tf.float32)
#         keep_prob2 = tf.placeholder(tf.float32)

        # 変数 (Valriables)
        W_h1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden1]))
        b_h1 = tf.Variable(tf.zeros([hidden1]))
        W_h2 = tf.Variable(tf.truncated_normal([hidden1, hidden2]))
        b_h2 = tf.Variable(tf.zeros([hidden2]))
#         W_h3 = tf.Variable(tf.truncated_normal([hidden2, hidden3]))
#         b_h3 = tf.Variable(tf.zeros([hidden3]))
        W_out = tf.Variable(tf.truncated_normal([hidden2, num_labels]))
        b_out = tf.Variable(tf.zeros([num_labels]))

        # トレーニングの計算 (Training computation)
        act_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, W_h1) + b_h1)
        act_h2 = tf.nn.relu(tf.matmul(act_h1, W_h2) + b_h2)
        act_h2_drop = tf.nn.dropout(act_h2, keep_prob1)
#         act_h3 = tf.nn.relu(tf.matmul(act_h2_drop, W_h3) + b_h3)
#         act_h3_drop = tf.nn.dropout(act_h3, keep_prob2)
        logits = tf.matmul(act_h2_drop, W_out) + b_out
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

        # 最適化(Optimizer.)
    #     # 引数: 1:学習率の初期設定, 2:現在のstep数, 3:学習がどの程度進んだら学習量を減衰させるか,4:減衰させる量
    #     global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    #     learning_rate = tf.train.exponential_decay(0.01, global_step, 10000, 0.95)
    #     optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        optimizer = tf.train.AdamOptimizer().minimize(loss)

        # トレーニング、バリデーション、テストデータに対しての予測
        train_prediction = tf.nn.softmax(logits)

        val_act_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, W_h1) + b_h1)
        val_act_h2 = tf.nn.relu(tf.matmul(val_act_h1, W_h2) + b_h2)
#         val_act_h3 = tf.nn.relu(tf.matmul(val_act_h2, W_h3) + b_h3)
        val_logits = tf.matmul(val_act_h2, W_out) + b_out
    #     val_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(val_logits, tf_valid_labels))
        valid_prediction = tf.nn.softmax(val_logits)

        test_act_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, W_h1) + b_h1)
        test_act_h2 = tf.nn.relu(tf.matmul(test_act_h1, W_h2) + b_h2)
#         test_act_h3 = tf.nn.relu(tf.matmul(test_act_h2, W_h3) + b_h3)
        test_logits = tf.matmul(test_act_h2, W_out) + b_out
        test_prediction = tf.nn.softmax(test_logits)

    
    print('keep_prob1:', keep_prob_val1)
    
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("変数を初期化しました")

        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            # ミニバッチの作成
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob1: keep_prob_val1 }
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            if (step % 2000 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

keep_prob1: 1.0
変数を初期化しました
Minibatch loss at step 0: 6522.518555
Minibatch accuracy: 15.6%
Validation accuracy: 15.4%
Minibatch loss at step 2000: 134.483856
Minibatch accuracy: 86.3%
Validation accuracy: 84.1%
Minibatch loss at step 4000: 55.538170
Minibatch accuracy: 91.4%
Validation accuracy: 84.8%
Minibatch loss at step 6000: 36.211388
Minibatch accuracy: 93.4%
Validation accuracy: 85.2%
Minibatch loss at step 8000: 26.444284
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
Minibatch loss at step 10000: 17.560623
Minibatch accuracy: 95.3%
Validation accuracy: 86.1%
Minibatch loss at step 12000: 7.348927
Minibatch accuracy: 96.1%
Validation accuracy: 86.7%
Minibatch loss at step 14000: 9.772429
Minibatch accuracy: 96.5%
Validation accuracy: 87.1%
Minibatch loss at step 16000: 6.312603
Minibatch accuracy: 96.5%
Validation accuracy: 86.7%
Minibatch loss at step 18000: 12.030708
Minibatch accuracy: 96.5%
Validation accuracy: 87.3%
Minibatch loss at step 20000: 1.726240
Minibatch ac